# Test: Create & Upload Batch of SAEF Datasets

## About
Interactive test of upload of a batch of SAEF digital objects in an existing inventory
- **Created:** 2023/01/03
- **Last update:** 2023/01/04

## Globals
Define global variables for testing purposes.

In [ ]:
g_saef_module_path = '../src'

# this inventory already contains the required saef mms_id and object_tags fields
# the inventory also contains proper file_path values
g_test_saef_inventory = './inventory/test_saef_updated_inventory.csv'

# demo.dataverse.org API key
g_demo_dataverse_api_key = ''

## Modules

Add local modules path to Jupyter system path

In [ ]:
import sys
if g_saef_module_path not in sys.path:
    sys.path.append(g_saef_module_path)

In [ ]:
import lcd
import saef
import collection
import pprint

### 1. Load the `SAEFProjectConfig`

In [ ]:
# create a saef project config instance
config = saef.SAEFProjectConfig()

# test a valid ini file
print('SAEFProjectConfig::read_ini: {}'.format(config.read_ini('./config/test_saef_config_true.ini')))

# print initialization status
print('SAEFProjectConfig::initd: {}'.format(config.initd()))

# get configuration options
options = config.get_options()
pprint.pprint('SAEFProjectConfig::get_options: {}'.format(options),sort_dicts=True)

### 2. Load the test inventory 

In [ ]:
# create file inventory instance
fi = lcd.FileInventory()

# read inventory from a csv file
print ('FileInventory::from_file: {}'.format(fi.from_file(g_test_saef_inventory)))

# get the owner supplied names from the inventory
object_osns = fi.get_owner_supplied_names()

print ('Owner-supplied names: {}'.format(len(object_osns)))
pprint.pprint(object_osns)

### 3. For each owner-supplied name, create a `SAEFDigitalObject`

In [ ]:
# dictionary of saef digital objects
digital_objects = {}

for osn in object_osns:
    # get the files associated with the osn
    files_df = fi.get_files('object_osn',osn)

    # create saef digital object instance
    saefdo = saef.SAEFDigitalObject()

    # read saef digital object from dataframe
    if (saefdo.from_dataframe(files_df) == True): 
        # add digital object to dictionary
        digital_objects[osn] = saefdo
    else:
        print('Warning: failed to create SAEFDigitalObject for: {}'.format(osn))

### 4. For each `SAEFDigitalObject` create a `SAEFDataset`

In [ ]:
datasets = {}

for osn in object_osns:
    # create/initialize the dataset instance
    saefdataset = saef.SAEFDataset()
    saefdo = digital_objects[osn]
    if (saefdataset.initialize(saefdo, config) == True):
        datasets[osn] = saefdataset
    else:
        print('Warning: failed to create SAEFDataset for: {}'.format(osn))
        
pprint.pprint(datasets)

### 5. Initialize the `pyDataverse` API

In [ ]:
# import pyDataverse packages
from pyDataverse.api import NativeApi

# create pyDataverse API adapter
dataverse_collection_url = options.get('dataverse').get('dataverse_collection_url')
dataverse_installation_url = options.get('dataverse').get('dataverse_installation_url')

# use the locally set dataverse api key
api = NativeApi(dataverse_installation_url, g_demo_dataverse_api_key)
print('{}'.format(api))

### 6. For each `SAEFDataset`, create a dataset on `demo.dataverse.org` and upload its datafiles
- Note: Be certain to **TURN OFF** the SAEF custom metadata block before doing so, otherwise the API calls will fail.

In [ ]:
for dataset in datasets.keys():
    # get the saef dataset instance
    d = datasets[dataset]
    # create the dataset on demo.dataverse.org
    print('Creating dataset for: {}'.format(dataset))
    if (d.create(api) == False):
        print ('Error: Failed to create dataset on demo.dataverse.org: {}'.format(dataset))
    else:
        # upload the datafiles
        if (d.direct_upload_datafiles(api) == False):
            print ('Error: Failed to upload datafiles for: {}'.format(dataset))
        else:
            # upload relationship files
            if (d.direct_upload_relationships(api) == False):
                print ('Error: Failed to upload relationship files for: {}'.format(dataset))
    # all operations succeeded
    print('Successfully created and uploaded dataset content: {}'.format(dataset))
    print('#####')

### 7. Apply custom SAEF metadata for all datasets
- Note: First, make sure to **TURN ON** the SAEF custom metadata block on the demo dataverse installation
- Note: 2023/01/03: Currently, the demo installation appears to be in a strange state where two conflicting versions of the SAEF custom metadata block appear to be active. That situation needs to be corrected before datasets can be updated with the SAEF custom metadata.

In [ ]:
for dataset in datasets.keys():
    # get the saef dataset instance
    d = datasets[dataset]
    # get the custom metadata
    md = d.get_dataset_metadata()
    custom_md = md.get('dataset').get('customSAEF')
    # update the custom metadata
    if (d.upload_saef_metadata(api, custom_md) == False):
        print('Error: Failed to update the custom SAEF metadata for: {}'.format(dataset))
    else:
        print('Successfully updated custom SAEF metadata for: {}'.format(dataset))

### Clean up `demo.dataverse.org` collection
Delete the datasets from demo.dataverse.org/saef

In [ ]:
for dataset in datasets.keys():
    # get the saef dataset instance
    d = datasets[dataset]
    # get the dataset pid
    pid = d.get_dataset_pid()
    # destroy the dataset
    response = api.destroy_dataset(pid, is_pid=True, auth=True)
    status = response.json().get('status')
    print('SAEFDataset::destroy dataset: {}'.format(status))    

**End document.**